In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


download_path = '/Users/jonathanoh/Desktop/down'
script_dir = '/Users/jonathanoh/Desktop/script/crawling'
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# script_dir = "/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/crawling"
# download_path = "/Users/jonathanoh/Desktop/contents/exiftoexcel/temp_image_zip_treated"
# s = Service('/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/chromedriver-mac-arm64/chromedriver')





def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv') :
            os.remove(os.path.join(directory_path, file))
    csv_file = os.path.join(directory_path, "url_data.csv")

    
    combined_df.to_csv(csv_file, index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기
    i = 0

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                i+=1
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                title = title.replace("/", "_")
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))


        
            
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
#         if i==0:
#             return True
#         else :
#             return False  # 오류 발생시 실패로 간주

    if len(failed_downloads)==0 or i == 0 :
        return True
    else : 
        return False  
    



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    
    duplicate_url = combine_csv_files_remove_duplicates(script_dir)
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urlsx = duplicate_url.loc[:, 'url'].to_list()
    old_urls = []
    for x in old_urlsx :
        old_urls.append(x.split('?')[0])

    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href.split('?')[0] in old_urls : 
                mm = href.split('?')[0].split('/')[-1]
                print(f"Processing: {total} 중 {i} {mm}__{title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href.split('?')[0], title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [32]:
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'


options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)

driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')



# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

In [33]:
post_list = []
for i in range(5):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)

post_list.reverse()
post_list

[('https://arca.live/b/aiartreal/95915784?p=5', '엉큰녀', '목소리로일하자'),
 ('https://arca.live/b/aiartreal/95916569?p=5',
  '알몸 앞치마 - 반실성인 체크 받으러 왔습니다.',
  '저기있는곰'),
 ('https://arca.live/b/aiartreal/95919325?p=5',
  'PurelyWholesome_v0.6 찍먹',
  'adoniel'),
 ('https://arca.live/b/aiartreal/95919981?p=5', '키보드야 기싱꿍꼬또?', '정배통프살'),
 ('https://arca.live/b/aiartreal/95920660?p=5',
  '에반게리온 - 아스카 (PurelyWholesome_v0.6)',
  '레드프레임21'),
 ('https://arca.live/b/aiartreal/95921071?p=5', '오빠 나 어때??...(스압)', '실사좋아'),
 ('https://arca.live/b/aiartreal/95921676?p=5', '미래의 사이버펑크 라텍스', '리버스Q'),
 ('https://arca.live/b/aiartreal/95922544?p=5', '너무 말랐다..', 'oooi'),
 ('https://arca.live/b/aiartreal/95922995?p=5',
  '밤일전 손이 은근히 많이 가는 (?)',
  '실사좋아'),
 ('https://arca.live/b/aiartreal/95923251?p=5',
  'PurelyWholesome_v0.1.fp16 수영복 찍먹',
  'carmen'),
 ('https://arca.live/b/aiartreal/95923525?p=5',
  'PurelyWholesome_v0.2.fp16 수영복 찍먹(계속)',
  'carmen'),
 ('https://arca.live/b/aiartreal/95923736?p=5',
  'PurelyWholesome_v

In [39]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

중복 : 엉큰녀
중복 : 알몸 앞치마 - 반실성인 체크 받으러 왔습니다.
중복 : PurelyWholesome_v0.6 찍먹
중복 : 키보드야 기싱꿍꼬또?
중복 : 에반게리온 - 아스카 (PurelyWholesome_v0.6)
중복 : 오빠 나 어때??...(스압)
중복 : 미래의 사이버펑크 라텍스
중복 : 너무 말랐다..
중복 : 밤일전 손이 은근히 많이 가는 (?)
중복 : PurelyWholesome_v0.1.fp16 수영복 찍먹
중복 : PurelyWholesome_v0.2.fp16 수영복 찍먹(계속)
중복 : PurelyWholesome_v0.3.fp16 수영복 찍먹입니다.
중복 : 여자
중복 : PurelyWholesome_v0.5.fp16 수영복 찍먹(앞으로 한개 더 남았네...)
중복 : PurelyWholesome_v0.6.fp16 수영복 찍먹(마지막)
중복 : 문문문님의 5moonAisan_v1A 수영복 찍먹
중복 : 5moonAisan_v1B 수영복 찍먹
중복 : 사부작 사부작..
중복 : 5moonAisan_v1C 수영복 찍먹입니다.
중복 : hanimix_v19(2.5D_nsfw)外 4개 모델 공유/ 종료
중복 : 육상선수
중복 : 조선 무사 복장과 미래 세계의 조합~
중복 : PurelyWholesome_v0.6 모델입니다.
중복 : 네 발 포즈
중복 : 은과 금
중복 : 크리스마스 피규어
중복 : 산타걸
중복 : T2I 고양이손 해드렸습니다.
중복 : 스트리트 파이터 - 캐미 화이트 코스프레
중복 : 근육)보디빌딩 사무실의 여사무원
중복 : [성공기] T2I 성공 ! 첫빠인가 !
중복 : 기타소녀 실사판
중복 : 얼빠샷
중복 : 간호사
중복 : 타인의 취향...(feat 주딱)
중복 : bottomless
중복 : 슬프다
중복 : 얼뺘샷-2(눈을 감고)
중복 : [PurelyWholesome_v0.7.3] 유배가기 좋은 모델
중복 : 바이올린 (hanimix_v19(2.5D_nsfw), hanimix_v19(2.6D_nsfw), ha

In [20]:
list

[('https://arca.live/b/aiartreal/95868029?p=1',
  '로컬 SD 새로깔았더니 deepdanbooru 사라져서 안보이는데 어떻게 함?',
  '헥사'),
 ('https://arca.live/b/aiartreal/95870754?p=1',
  '포토샵 - 너무 매끈한 피부에 질감입히기',
  '냥이집사')]

In [10]:
len(list)

14

In [35]:
list = process_images(driver, list, download_path)

Processing: 16 중 1 95983772__저도 성형수술 함시켜봄요  _  크런키z
failed : https://arca.live/b/aiartreal/95983772?p=2
Processing: 16 중 2 95986549__작심삼일 일주일 경과  _  냥이집사
failed : https://arca.live/b/aiartreal/95986549?p=2
Processing: 16 중 3 95998151__안녕하세요 bmab를 공부해보고 있는데 질문이 있습니다  _  neewa
failed : https://arca.live/b/aiartreal/95998151?p=2
Processing: 16 중 4 95999419__sd-dynamic-prompts 사용하는 분들 템플릿을 메타데이터에 저장 옵션 사용 시 버그 주의  _  thec
failed : https://arca.live/b/aiartreal/95999419?p=1
Processing: 16 중 5 96000037__sdxl 모델에 sd1.5 머저 블록 가능한가요?  _  spearmint
failed : https://arca.live/b/aiartreal/96000037?p=1
Processing: 16 중 6 96000901__메카  _  빠다파리
Processing: 16 중 7 96003517__엡신스 이게 요~~~물이구마이  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/96003517?p=1
Processing: 16 중 8 96003555__역시 인물 뽑을땐 비슷한 인물 넣는게 최고긴하네...  _  비스비스
failed : https://arca.live/b/aiartreal/96003555?p=1
Processing: 16 중 9 96006804__온천  _  강해린
Processing: 16 중 10 96009384__모델 공유합니다 - TPix_typeURe_v1.31L  _  TP
failed : https://ar

In [27]:
list = process_images(driver, list, download_path)

Processing: 26 중 1 95889582__국내 생성 AI 영화 '저작권 첫 인정'...세계 2번째 사례  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/95889582?p=5
Processing: 26 중 2 95891051__경관 (와카 포함)  _  빠다파리
failed : https://arca.live/b/aiartreal/95891051?p=5
Processing: 26 중 3 95894529__성인 그림이 반실성인과 통합되고 영상 생김  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/95894529?p=4
Processing: 26 중 4 95896635__로봇걸  _  빠다파리
failed : https://arca.live/b/aiartreal/95896635?p=4
Processing: 26 중 5 95904302__t3_Ver10 모델입니다.  _  badabebe
failed : https://arca.live/b/aiartreal/95904302?p=4
Processing: 26 중 6 95905182__비누거품속의 소녀  _  빠다파리
failed : https://arca.live/b/aiartreal/95905182?p=4
Processing: 26 중 7 95907758__오랜만에 한장  _  oooi
Processing: 26 중 8 95912511__hanimix_v19B(nsfw) 찍먹 "무시무시한 지진 공포의 현장"입니다.  _  ghana
Processing: 26 중 9 95914235__오늘도 한 수 배웠다.  _  이노프리
failed : https://arca.live/b/aiartreal/95914235?p=4
Processing: 26 중 10 95930640__[성공기] T2I 성공 ! 첫빠인가 !  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/

In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [36]:
temp = []
for url, title, user in list :
    temp.append([url.split('?')[0], title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'
csv_fail_fiile = os.path.join(script_dir, 'fail_list.csv')
df.to_csv(csv_fail_fiile, index=False)
df = combine_csv_files_remove_duplicates(script_dir)

In [37]:
df = combine_csv_files_remove_duplicates(script_dir)

In [38]:
df

,url,filename
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4931442.jpeg
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4924345.jpeg
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926329.jpeg
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926315.jpeg
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926328.jpeg
...,...,...
4835,https://arca.live/b/aiartreal/96003517,엡신스 이게 요~~~물이구마이
4836,https://arca.live/b/aiartreal/96003555,역시 인물 뽑을땐 비슷한 인물 넣는게 최고긴하네...
4837,https://arca.live/b/aiartreal/96009384,모델 공유합니다 - TPix_typeURe_v1.31L
4838,https://arca.live/b/aiartreal/96009415,캠핑걸 & 유니폼


In [11]:
k = [list[5]]
del list[5]

In [52]:
df['url'] = df['url'].apply(lambda x: x.split('?')[0])

In [53]:
df.to_csv(os.path.join(script_dir, "url_datas.csv"), index=False)

# SAMPLING

In [42]:
import os
import zipfile
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from datetime import datetime

# 현재 날짜와 시간을 가져옵니다
now = datetime.now()

# 현재 날짜를 "년-월-일" 형식의 문자열로 변환합니다
current_date = now.strftime("%Y%m%d")

def zip_files(files, zip_name, extract_dir):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for file in files:
            abs_path = os.path.join(extract_dir, file)
            if not os.path.exists(abs_path):
                print(f"File not found, skipping: {abs_path}")
                continue
            print(f"Zipping file: {abs_path}")
            zipf.write(abs_path, os.path.basename(file))
            os.remove(abs_path)


def extract_first_image(zip_dir, extract_dir):
    image_formats = ('.jpg', '.jpeg', '.png')
    extracted_files = []

    for filename in os.listdir(zip_dir):
        if filename.endswith('.zip') and not filename.lower().startswith('civitai') :
            zip_path = os.path.join(zip_dir, filename)
            if not zipfile.is_zipfile(zip_path):
                print(f"Skipping non-ZIP file: {filename}")
                continue
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                for file in zip_ref.namelist():
                    if file.lower().endswith(image_formats):
                        abs_path = os.path.join(extract_dir, file)
                        print(f"Extracting file: {abs_path}")  # Diagnostic print
                        zip_ref.extract(file, extract_dir)
                        extracted_files.append(file)
                        break

    if extracted_files:
        # Extract the folder name from the extract_dir path
        folder_name = os.path.basename(os.path.normpath(zip_dir))
        zip_file_name = f"{current_date}_sampling.zip"
        zip_files(extracted_files, os.path.join(extract_dir, zip_file_name), extract_dir)
        print("완료 되었습니다. 오정훈님")


def select_zip_folder():
    zip_dir = filedialog.askdirectory(title="Select ZIP Folder")
    if zip_dir:
        extract_dir = filedialog.askdirectory(title="Select samling Folder")
        if extract_dir :
            extract_first_image(zip_dir, extract_dir)

# GUI setup
# root = tk.Tk()
# root.title("ZIP Extractor")

# select_button = tk.Button(root, text="Select ZIP Folder and Extract", command=select_zip_folder)
# select_button.pack(pady=20)

# root.mainloop()


In [43]:
arca = '/Users/jonathanoh/Desktop/down/arca_0107'
extract_first_image(arca, download_path)



Extracting file: /Users/jonathanoh/Desktop/down/[2D] 테마별 몇장씩 모음_91.png
Extracting file: /Users/jonathanoh/Desktop/down/5moonAisan_v1B 수영복 찍먹_115.png
Extracting file: /Users/jonathanoh/Desktop/down/NeMix MuSS-T v1.25.0 A1111 사용 찍먹_83.jpg
Extracting file: /Users/jonathanoh/Desktop/down/hanimix_real_v16B 서술형 와카로 찍먹 해봅니다._88.png
Extracting file: /Users/jonathanoh/Desktop/down/2d 컨셉 ( 2d 로라 + hanimix_real_v16A)_81.png
Extracting file: /Users/jonathanoh/Desktop/down/토끼소녀_86.jpg
Extracting file: /Users/jonathanoh/Desktop/down/hanimix_real_v16B 수영복 찍먹_115.png
Extracting file: /Users/jonathanoh/Desktop/down/해변 셀카 (TPix_typeRExp_v1.35L)_82.png
Extracting file: /Users/jonathanoh/Desktop/down/XL 랜덤 와카_86.jpg
Extracting file: /Users/jonathanoh/Desktop/down/hanimix_real_v16B 테스트_82.png
Extracting file: /Users/jonathanoh/Desktop/down/기계 갑옷_88.jpg
Extracting file: /Users/jonathanoh/Desktop/down/타임머신_84.jpg
Extracting file: /Users/jonathanoh/Desktop/down/오늘 조합해본 얼굴들_85.png
Extracting file: /Users/jonat

Extracting file: /Users/jonathanoh/Desktop/down/(SDXL) 드랍 더 비ㅌ-!!_88.png
Extracting file: /Users/jonathanoh/Desktop/down/미완성 모델 + 달잡이님 프롬프트 줍?줍_88.png
Extracting file: /Users/jonathanoh/Desktop/down/NewHorizon_83.png
Extracting file: /Users/jonathanoh/Desktop/down/애착모델( TPix_typeUR_v0.2_fp16) 판타지+메카닉_88.png
Extracting file: /Users/jonathanoh/Desktop/down/파판7 리메이크 티파 실사버전_87.png
Extracting file: /Users/jonathanoh/Desktop/down/메카닉 컨셉 ( hanimix_real_v16A)_82.png
Extracting file: /Users/jonathanoh/Desktop/down/Enchanter_81.jpg
Extracting file: /Users/jonathanoh/Desktop/down/막뽑_81.png
Extracting file: /Users/jonathanoh/Desktop/down/A vibrant city_88.png
Extracting file: /Users/jonathanoh/Desktop/down/늑대 (hanimix_v18A)_80.png
Extracting file: /Users/jonathanoh/Desktop/down/28 내믹스NeMix MuSS-N 얼짱각도_80.png
Extracting file: /Users/jonathanoh/Desktop/down/키보드야 기싱꿍꼬또?_81.png
Extracting file: /Users/jonathanoh/Desktop/down/네뷸라 프린팅 드레스 (GAMBLE-MIX-REAL-35, GAMBLE-MIX-REAL-36)_80.png
Extracting file:

Extracting file: /Users/jonathanoh/Desktop/down/오늘 마지막 테스트 (Cocktail Dress)_108.jpg
Extracting file: /Users/jonathanoh/Desktop/down/dalcefo_XL-PhotoBase-TB-Test_어쩌자고 이 엄청난 모델을...._80.png
Extracting file: /Users/jonathanoh/Desktop/down/hanimix_real_v16_81.png
Extracting file: /Users/jonathanoh/Desktop/down/2.5d 모델 ( hash: 4cb6bd041b, Model: nigi3d_v20)_89.png
Extracting file: /Users/jonathanoh/Desktop/down/super-hentai 찍먹_81.png
Extracting file: /Users/jonathanoh/Desktop/down/딸기 케이크 (PurelyWholesome_v0.4)_80.png
Extracting file: /Users/jonathanoh/Desktop/down/hanimix_real_v16A 부먹_88.jpg
Extracting file: /Users/jonathanoh/Desktop/down/whale hunt in the 19th_82.jpg
Extracting file: /Users/jonathanoh/Desktop/down/2d 판타지 - 마법영창 컨셉_83.png
Extracting file: /Users/jonathanoh/Desktop/down/얼빠샷_83.png
Extracting file: /Users/jonathanoh/Desktop/down/SDXL 피쉬볼 헤드_82.jpg
Extracting file: /Users/jonathanoh/Desktop/down/이정도면 모델 사진으로 쓸수 있으려나!!_81.png
Extracting file: /Users/jonathanoh/Desktop/down/hanim

Extracting file: /Users/jonathanoh/Desktop/down/터틀넥 + RGTA 프롬 조합_89.jpg
Extracting file: /Users/jonathanoh/Desktop/down/오랜만에 올리는 BBW!~!_79.png
Extracting file: /Users/jonathanoh/Desktop/down/dalcefo_XL-PhotoBase-TB-Test 얼빠 안정화_80.png
Extracting file: /Users/jonathanoh/Desktop/down/hanimix_real_v16 찍먹_80.png
Extracting file: /Users/jonathanoh/Desktop/down/[PurelyWholesome_v0.7.3] 마지막_82.jpg
Extracting file: /Users/jonathanoh/Desktop/down/서술형 랜덤와카 2_84.jpg
Extracting file: /Users/jonathanoh/Desktop/down/re: 이것좀 해"줘"_81.png
Extracting file: /Users/jonathanoh/Desktop/down/롤리타 드레스_88.jpg
Extracting file: /Users/jonathanoh/Desktop/down/패션 컨셉 2d 그림_94.png
Extracting file: /Users/jonathanoh/Desktop/down/얼굴을 만들어봅시다 feat. hanimix_real_v16A_82.png
Extracting file: /Users/jonathanoh/Desktop/down/비누 거품속의 소녀_81.jpg
Extracting file: /Users/jonathanoh/Desktop/down/태권도하는 스튜어디스 눈나 feat.빤스_80.png
Extracting file: /Users/jonathanoh/Desktop/down/유로피언 비키니 우먼_88.jpg
Extracting file: /Users/jonathanoh/Desktop

Extracting file: /Users/jonathanoh/Desktop/down/색존신고 간만에 짤 투척 ㅌㅌㅌ_89.png
Extracting file: /Users/jonathanoh/Desktop/down/2.5D입니다._98.png
Extracting file: /Users/jonathanoh/Desktop/down/아오 더럽게 만들기 힘드네_80.png
Extracting file: /Users/jonathanoh/Desktop/down/과거의 유물..._80.png
Extracting file: /Users/jonathanoh/Desktop/down/막뽑_88.png
Extracting file: /Users/jonathanoh/Desktop/down/문문문님의 5moonAisan_v1A 수영복 찍먹_97.png
Extracting file: /Users/jonathanoh/Desktop/down/어우씨 어렵네 이거 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ' 이것좀 해"줘" '_80.jpg
Extracting file: /Users/jonathanoh/Desktop/down/hanimix_XL_v11Ac 테스트_81.png
Extracting file: /Users/jonathanoh/Desktop/down/(2D) Nami X Orc (94shot)_134.jpg
Extracting file: /Users/jonathanoh/Desktop/down/(스압) see-through swimsuit_80.png
Extracting file: /Users/jonathanoh/Desktop/down/on off_89.jpg
Extracting file: /Users/jonathanoh/Desktop/down/동물을 사랑하는 소녀_85.jpg
Extracting file: /Users/jonathanoh/Desktop/down/헬스장_80.jpg
Extracting file: /Users/jonathanoh/Desktop/down/반실 모델을 만들어본 이유 (문제시삭제바랍

...

Zipping file: /Users/jonathanoh/Desktop/down/그래픽카드 고민 가성비 와 끝판왕 스테블디퓨전 밴치자료 분석_81.png
Zipping file: /Users/jonathanoh/Desktop/down/hanimix_real_v14+ 노출없이 감성 느낌만_84.png
Zipping file: /Users/jonathanoh/Desktop/down/TPix_typeUR_v1.3_fp16 찍먹] 쥬디홉스 로라의 당근 토끼요정_80.png
Zipping file: /Users/jonathanoh/Desktop/down/돌핀 팬츠 - 문제시 삭제_88.png
Zipping file: /Users/jonathanoh/Desktop/down/새해 복 많이 받으세요_79.png
Zipping file: /Users/jonathanoh/Desktop/down/(스압) 高-고-Go!!_109.png
Zipping file: /Users/jonathanoh/Desktop/down/슬프다_83.png
Zipping file: /Users/jonathanoh/Desktop/down/롱 스웨터_88.jpg
Zipping file: /Users/jonathanoh/Desktop/down/갑자기 꿈 속에 나타난 서큐버스_81.png
Zipping file: /Users/jonathanoh/Desktop/down/초상화_88.jpg
Zipping file: /Users/jonathanoh/Desktop/down/GAMBLE-MIX-REAL-40 _ 찍먹_88.png
Zipping file: /Users/jonathanoh/Desktop/down/자기전에 한장_80.jpg
Zipping file: /Users/jonathanoh/Desktop/down/세팅 값 찾다가 포기..._82.png
Zipping file: /Users/jonathanoh/Desktop/down/반투명_89.jpg
Zipping file: /Users/jonathanoh/Desktop

Zipping file: /Users/jonathanoh/Desktop/down/자러가기전 업로드_80.png
Zipping file: /Users/jonathanoh/Desktop/down/악마 테마_79.jpg
Zipping file: /Users/jonathanoh/Desktop/down/내년엔 다들 부자되세여~_81.jpg
Zipping file: /Users/jonathanoh/Desktop/down/무제_80.png
Zipping file: /Users/jonathanoh/Desktop/down/강아지들_88.jpg
Zipping file: /Users/jonathanoh/Desktop/down/풍등_88.jpg
Zipping file: /Users/jonathanoh/Desktop/down/hanmix_19B 찍먹을 해봤는데요 BAMB에서 뭔가를 건드렸는데_80.png
Zipping file: /Users/jonathanoh/Desktop/down/[스압] SDXL 은 당분간 포기 SD 1.5 가 더 나음_144.jpg
Zipping file: /Users/jonathanoh/Desktop/down/콘서트장에 온 티파_85.png
Zipping file: /Users/jonathanoh/Desktop/down/쌩초보^^;; 등판 과다 노출 로 병맛_79.png
Zipping file: /Users/jonathanoh/Desktop/down/주말~!!_81.jpg
Zipping file: /Users/jonathanoh/Desktop/down/ARAZmixx_020 찍먹. (Feat. 옵치솜장인님 로라)_98.png
Zipping file: /Users/jonathanoh/Desktop/down/hanimix_v19B(nsfw) 찍먹 춤바람입니다._88.png
Zipping file: /Users/jonathanoh/Desktop/down/AnimateDiff 찍먹..._82.png
Zipping file: /Users/jonathanoh/Deskt

Zipping file: /Users/jonathanoh/Desktop/down/PurelyWholesome_v0.6 . . ._80.png
Zipping file: /Users/jonathanoh/Desktop/down/아침 과제 시작 랜덤짤_104.jpg
Zipping file: /Users/jonathanoh/Desktop/down/새얼굴 + 신년엔 동탄_89.jpg
File not found, skipping: /Users/jonathanoh/Desktop/down/고통의 골반바지_82.png
Zipping file: /Users/jonathanoh/Desktop/down/육덕_80.png
Zipping file: /Users/jonathanoh/Desktop/down/12월달 수익 결산_80.png
Zipping file: /Users/jonathanoh/Desktop/down/지휘관_80.jpg
Zipping file: /Users/jonathanoh/Desktop/down/선택장애_85.png
Zipping file: /Users/jonathanoh/Desktop/down/새해복 많이 받으세요~_104.jpg
Zipping file: /Users/jonathanoh/Desktop/down/swimwaer 테스트샷_89.jpg
Zipping file: /Users/jonathanoh/Desktop/down/NeMix MuSS-T v1.25.0 A1111 찍먹 바닷가_88.jpg
Zipping file: /Users/jonathanoh/Desktop/down/[모델공유] GAMBLE-MIX-REAL_83.png
Zipping file: /Users/jonathanoh/Desktop/down/코르셋_81.jpg
Zipping file: /Users/jonathanoh/Desktop/down/일상복_86.png
Zipping file: /Users/jonathanoh/Desktop/down/우주공간_88.jpg
Zipping file: /Users/jon

File not found, skipping: /Users/jonathanoh/Desktop/down/젖_80.jpg
Zipping file: /Users/jonathanoh/Desktop/down/야외 출사 컨셉_88.jpg
Zipping file: /Users/jonathanoh/Desktop/down/원하는거 다 이루시는 새해 되세요_89.png
Zipping file: /Users/jonathanoh/Desktop/down/저도 골반바지^^;;;; darkq0님 t3-v10버전....(스압)_88.jpg
Zipping file: /Users/jonathanoh/Desktop/down/2024년 축전~_79.png
Zipping file: /Users/jonathanoh/Desktop/down/hanimix_v19(2.5D_nsfw) 수영복 찍먹_122.png
Zipping file: /Users/jonathanoh/Desktop/down/오늘은 비교적 멀쩡한 그림 가져왔어요._79.png
Zipping file: /Users/jonathanoh/Desktop/down/5moon 모델 찍먹. (Feat. 옵치솜장인님 로라)_96.png
Zipping file: /Users/jonathanoh/Desktop/down/배가본드_90.jpg
Zipping file: /Users/jonathanoh/Desktop/down/....._80.png
Zipping file: /Users/jonathanoh/Desktop/down/Sex and Pleasure_84.png
Zipping file: /Users/jonathanoh/Desktop/down/뽑다가 느낌이 좋아서 한장_80.jpg
Zipping file: /Users/jonathanoh/Desktop/down/에어 리스_89.jpg
Zipping file: /Users/jonathanoh/Desktop/down/찢어진 청바지를 입고 출근하는 여자들_84.png
Zipping file: /Users/jonath

Zipping file: /Users/jonathanoh/Desktop/down/춘-리 만들어 봤읍니다_94.png
Zipping file: /Users/jonathanoh/Desktop/down/사진같이 나와서 ^^ (1장)_79.png
Zipping file: /Users/jonathanoh/Desktop/down/대충 막 뽑은 사진_88.png
Zipping file: /Users/jonathanoh/Desktop/down/내가 원한건 비키니 탑에 청반바지 였는데_89.png
Zipping file: /Users/jonathanoh/Desktop/down/연리지_80.png
Zipping file: /Users/jonathanoh/Desktop/down/"탈출성공"_81.jpg
Zipping file: /Users/jonathanoh/Desktop/down/드디어 글카가 죽었습니다._80.png
Zipping file: /Users/jonathanoh/Desktop/down/2d 모델 새로 만들어서 공유해봅니다_89.png
Zipping file: /Users/jonathanoh/Desktop/down/테스트 모델 10번_98.png
Zipping file: /Users/jonathanoh/Desktop/down/T2I 고양이손 해드렸습니다._80.png
Zipping file: /Users/jonathanoh/Desktop/down/이노프리님 베타버전 찍먹_81.png
Zipping file: /Users/jonathanoh/Desktop/down/대나무숲_86.jpg
Zipping file: /Users/jonathanoh/Desktop/down/short shorts_87.png
File not found, skipping: /Users/jonathanoh/Desktop/down/서술형 랜덤와카 3_89.jpg
Zipping file: /Users/jonathanoh/Desktop/down/꿀벌 코스튬 (5moonAisan_v1A)_82.png
Zi

In [44]:
# filelist = []
for file in os.listdir(arca):
    if file.lower().endswith('.jpeg'):
        file_path= os.path.join(arca, file)
#         filelist.append(file)
        try :
            image = Image.open(file_path)
        except: 
#             print(file_path)
            correct_extension = ".mov"
            new_file_path = file_path.rsplit('.', 1)[0] + correct_extension
            os.rename(file_path, new_file_path)
#         file_path= os.path.join(download_path, file)
#         correct_file_extension(file_path)

In [79]:
import io
arca = '/Users/jonathanoh/Desktop/down/20240107_sampling'
temp =[]
for file in os.listdir(arca):
    if not file.lower().startswith('._'):
        filename = os.path.join(arca, file)
#         print(file)
        image = Image.open(filename)
#         image = Image.open(io.BytesIO(file))
        if 'exif' in image.info.keys() :
            print(image.info.keys())
            temp.append([file, image.info['exif']])
        elif 'parameter' in image.info.keys():
            temp.appned([file, image.info['parameter']] )
#             image_exif = str(image.info['parameters']).replace(b'\x00', b'')
#             temp.append([file, image_exif])
# temp
temp

dict_keys(['exif', 'dpi'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi'])
dict_keys(['exif', 'dpi'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dpi', 'photoshop', 'adobe', 'adobe_transform'])
dict_keys(['exif', 'dp

[['Enchanter_81.jpg',
  b'Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x01\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\x1a\x00\x00\x00\x00\x00\x01\x92\x86\x00\x07\x00\x00(r\x00\x00\x00(UNICODE\x00\x00(\x00(\x00f\x00r\x00o\x00m\x00 \x00f\x00r\x00o\x00n\x00t\x00)\x00)\x00,\x00\n\x00\n\x00b\x00l\x00o\x00w\x00o\x00u\x00t\x00,\x00b\x00r\x00o\x00w\x00n\x00 \x00h\x00a\x00i\x00r\x00,\x00 \x00\n\x00 \x00\n\x00g\x00r\x00e\x00y\x00 \x00e\x00y\x00e\x00s\x00,\x00 \x00l\x00a\x00r\x00g\x00e\x00_\x00p\x00u\x00p\x00i\x00l\x00,\x00 \x00(\x00p\x00a\x00l\x00e\x00_\x00F\x00a\x00c\x00e\x00:\x001\x00.\x002\x00)\x00,\x00\n\x00 \x00\n\x00(\x00c\x00o\x00w\x00b\x00o\x00y\x00s\x00h\x00o\x00t\x00:\x001\x00.\x002\x00)\x00 \x00,\x00\n\x00 \x00\n\x00\n\x00l\x00a\x00r\x00g\x00e\x00_\x00b\x00r\x00e\x00a\x00s\x00t\x00,\x00 \x00w\x00i\x00d\x00e\x00_\x00h\x00i\x00p\x00,\x00 \x00(\x00s\x00l\x00e\x00n\x00d\x00e\x00r\x00:\x001\x00.\x004\x00)\x00,\x00 \x00s\x00h\x00i\x00n\x00y\x00_\x00a\x00n\x00d\x00_\x00g\x00l\x00o\x00s\x00s\x00y\x0

In [82]:
rr =[]
for file, exif in temp :
    clean = exif.replace(b'\x00',b'')
    rr.append([file, clean])

In [98]:
tt = []
for file, exif in rr :
    a = str(exif).split('UNICODE')[-1]
    a = a.split('Version')[0]
    tt.append([file, a])
tt

[['Enchanter_81.jpg',
  '((from front)),\\n\\nblowout,brown hair, \\n \\ngrey eyes, large_pupil, (pale_Face:1.2),\\n \\n(cowboyshot:1.2) ,\\n \\n\\nlarge_breast, wide_hip, (slender:1.4), shiny_and_glossy_skin,\\n\\n(((Enchanter, flowing robes, enchanted jewelry, imbuing weapons with magical power, in a hidden glade, enchanting a sword.))),\\n\\nglassy translucence, vibrant illustrations, ultra realistic, wide angle, 8k, HDR, professional photoshoot, high quality photo, realistic photo, realistic shadows, detailed shadows, realistic proportions, film grain, raw photo,(front),(realistic, realistic_background, high_resolution, distinct_image:1.2), (extremely intricate, detailed light, detailed shadow:1.1), (F4, 1/800s, ISO 100, RAW),photo-realistic,\\n\\n<lora:FilmG2:0.3> <lora:Pussy_Alicia_v01:0.3>\\n<lora:pussy:0.3> <lora:GodPussy1 v2:0.1>\\n<lora:koreanDollLikeness:0.3> \\n \\n<lora:detail_slider_v4:1.0>\\nNegative prompt: EasyNegative, puffy eyes, (fused fingers:1.4), (too many finger

In [95]:
str(rr[10][-1]).split('UNICODE')[-1].replace('\\n','')

'(wide_shot:1.2), blurry_background, 2girls, (Kosovan mixed, kpop idol), (cute 25 age old Martiniquais girl), seductive_smile, (light_body), (Pose that attracts the opposite sex),BREAK((presenting)), masterpiece, best_quality, realistic, high-resolution, (detailed), (photograpy), (photorealistic), ((__custom/newtype-swimwaer__)),  cowboy_shot,  Negative prompt: (bad-artist:0.7), (worst quality, low quality:1.4), abs, bad hand:1.33, badhandv4, EasyNegative, ((NSFW))Steps: 20, Sampler: DPM++ 2M SDE Karras, CFG scale: 5, Seed: 1001111719, Size: 512x768, Model hash: e312705dc0, Model: PurelyWholesome_v0.6.fp16, VAE hash: f921fb3f29, VAE: vae-ft-mse-840000-ema-pruned.safetensors, Denoising strength: 0.4, Clip skip: 2, Wildcard prompt: "(wide_shot:1.2), blurry_background, 2girls, (Kosovan mixed, kpop idol), (cute 25 age old Martiniquais girl), seductive_smile, (light_body), (Pose that attracts the opposite sex),\\BREAK\\((presenting)), masterpiece, best_quality, realistic, high-resolution, (

In [99]:
dd  = pd.DataFrame(tt)

In [100]:
ddd = os.path.join(arca, '11212.xlsx')
dd.to_excel(ddd, index=False)

In [101]:
ddd

'/Users/jonathanoh/Desktop/down/20240107_sampling/11212.xlsx'

In [2]:
excel_file = '/Users/jonathanoh/Desktop/down/arca_230107.xlsx'

In [10]:
df = pd.read_excel(excel_file)

In [4]:
# df = df.drop_duplicates(subset='column_name')

,ZIP Filename,Image Filename,Metadata,positive,negative,steps,POSITIVE,NEGATIVE,SETTING
0,./95814378_야발야발.zip,[2D] 테마별 몇장씩 모음_91.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
1,./95814378_야발야발.zip,[2D] 테마별 몇장씩 모음_85.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
2,./95814378_야발야발.zip,[2D] 테마별 몇장씩 모음_84.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
3,./95814378_야발야발.zip,[2D] 테마별 몇장씩 모음_90.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
4,./95814378_야발야발.zip,[2D] 테마별 몇장씩 모음_86.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
8650,./95553128_빠다파리.zip,용의 해 새해복 많이 받으세요_86.jpg,No metadata found\n,no data,no data,no data,NaN,NaN,NaN
8651,./95553128_빠다파리.zip,용의 해 새해복 많이 받으세요_79.jpg,No metadata found\n,no data,no data,no data,NaN,NaN,NaN
8652,./95553128_빠다파리.zip,용의 해 새해복 많이 받으세요_84.jpg,No metadata found\n,no data,no data,no data,NaN,NaN,NaN
8653,./95553128_빠다파리.zip,용의 해 새해복 많이 받으세요_85.jpg,No metadata found\n,no data,no data,no data,NaN,NaN,NaN


In [5]:
df = df.dropna()

In [6]:
df

,ZIP Filename,Image Filename,Metadata,positive,negative,steps,POSITIVE,NEGATIVE,SETTING
95,./95641157_토군.zip,hanimix_real_v16B 서술형 와카로 찍먹 해봅니다._80.png,"parameters: (1female, solo:1.6, (Korean female...","parameters: (1female, solo:1.6, (Korean female...","Negative prompt:\n(easyNegative, ng_deepnegati...","Steps: 45, Sampler: DPM++\n2M SDE Karras, CFG ...","(1female, solo:1.6, (Korean female, korean mix...","(easyNegative, ng_deepnegative_v1_75t:1.2), (b...","Steps: 45, Sampler: DPM++ 2M SDE Karras, CFG s..."
422,./95684180_바코드.zip,잡샷 와이드_91.jpg,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,"Negative prompt: EasyNegative, (pubic\nhair:1....","Steps: 40, Sampler:\nDPM++ 2M Karras, CFG scal...","full_body, quiff,((little happy face)),light b...","EasyNegative, (pubic hair:1.5), puffy eyes, (f...","Steps: 40, Sampler: DPM++ 2M Karras, CFG scale..."
423,./95684180_바코드.zip,잡샷 와이드_85.jpg,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,"Negative prompt: EasyNegative, (pubic hair:1.5...","Steps:\n40, Sampler: DPM++ 2M Karras, CFG scal...","full_body, cornrows,((normal face)),dirty blon...","EasyNegative, (pubic hair:1.5), puffy eyes, (f...","Steps: 40, Sampler: DPM++ 2M Karras, CFG scale..."
424,./95684180_바코드.zip,잡샷 와이드_84.jpg,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,"Negative prompt: EasyNegative,\n(pubic hair:1....","Steps: 40, Sampler:\nDPM++ 2M Karras, CFG scal...","full_body, twin drills,((wondering face)),burg...","EasyNegative, (pubic hair:1.5), puffy eyes, (f...","Steps: 40, Sampler: DPM++ 2M Karras, CFG scale..."
425,./95684180_바코드.zip,잡샷 와이드_90.jpg,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,"Negative prompt: EasyNegative, (pubic\nhair:1....","Steps: 40, Sampler:\nDPM++ 2M Karras, CFG scal...","full_body, messy hair,((beaming_smile face)),m...","EasyNegative, (pubic hair:1.5), puffy eyes, (f...","Steps: 40, Sampler: DPM++ 2M Karras, CFG scale..."
...,...,...,...,...,...,...,...,...,...
8494,./95666114_비스비스.zip,평범한 인물샷_86.png,"parameters: instagram photo,goddess-like body,...","parameters: instagram photo,goddess-like body,...",Negative prompt: (CyberRealistic_Negative-neg:...,"Steps: 50, Sampler:\nDPM++ 2M Karras, CFG scal...","instagram photo,goddess-like body,real photos,...","(CyberRealistic_Negative-neg:0.8),mature,curvy...","Steps: 50, Sampler: DPM++ 2M Karras, CFG scale..."
8495,./95666114_비스비스.zip,평범한 인물샷_88.png,"parameters: instagram photo,goddess-like body,...",no data,no data,"Steps: 50, Sampler:\nDPM++ 2M Karras, CFG scal...","instagram photo,goddess-like body,real photos,...","(CyberRealistic_Negative-neg:0.8),mature,curvy...","Steps: 50, Sampler: DPM++ 2M Karras, CFG scale..."
8496,./95666114_비스비스.zip,평범한 인물샷_89.png,"parameters: instagram photo,goddess-like body,...",no data,no data,"Steps: 50, Sampler:\nDPM++ 2M Karras, CFG scal...","instagram photo,goddess-like body,real photos,...","(CyberRealistic_Negative-neg:0.8),mature,curvy...","Steps: 50, Sampler: DPM++ 2M Karras, CFG scale..."
8642,./95633838_강해린.zip,오래된 프롬 '매장에서' 꺼내보기_80.png,"parameters: (masterpiece, best quality, photo-...","parameters: (masterpiece, best quality, photo-...","Negative prompt: (worst quality, low quality,\...","Steps: 50,\nSampler: DPM++ SDE Karras, CFG sca...","(masterpiece, best quality, photo-realistic, u...","(worst quality, low quality, normal quality, l...","Steps: 50, Sampler: DPM++ SDE Karras, CFG scal..."


In [11]:
# df = df.drop_duplicates(subset='positive')
df = df.drop_duplicates(subset='ZIP Filename')
df

,ZIP Filename,Image Filename,Metadata,positive,negative,steps,POSITIVE,NEGATIVE,SETTING
0,./95814378_야발야발.zip,[2D] 테마별 몇장씩 모음_91.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
27,./95924893_carmen.zip,5moonAisan_v1B 수영복 찍먹_115.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
70,./95944426_thec.zip,NeMix MuSS-T v1.25.0 A1111 사용 찍먹_83.jpg,"jfif: 257\njfif_version: (1, 1)\njfif_unit: 0\...",no data,no data,no data,NaN,NaN,NaN
82,./95641157_토군.zip,hanimix_real_v16B 서술형 와카로 찍먹 해봅니다._88.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
98,./95435769_냥이집사.zip,2d 컨셉 ( 2d 로라 + hanimix_real_v16A)_81.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
8629,./95982119_aborim.zip,animatediff_80.gif,version: GIF89a\nbackground: 0\nloop: 0\ncomme...,no data,no data,"Steps: 30, Sampler: DPM++ 3M SDE Karras,\nCFG ...",NaN,"(low quality:1.4), lowres, cloned face, doll, ...","Steps: 30, Sampler: DPM++ 3M SDE Karras, CFG s..."
8630,./95855739_몬TDAI.zip,underwater_88.png,"dpi: (72.009, 72.009)\n\n",no data,no data,no data,NaN,NaN,NaN
8642,./95633838_강해린.zip,오래된 프롬 '매장에서' 꺼내보기_80.png,"parameters: (masterpiece, best quality, photo-...","parameters: (masterpiece, best quality, photo-...","Negative prompt: (worst quality, low quality,\...","Steps: 50,\nSampler: DPM++ SDE Karras, CFG sca...","(masterpiece, best quality, photo-realistic, u...","(worst quality, low quality, normal quality, l...","Steps: 50, Sampler: DPM++ SDE Karras, CFG scal..."
8644,./95553128_빠다파리.zip,용의 해 새해복 많이 받으세요_88.jpg,No metadata found\n,no data,no data,no data,NaN,NaN,NaN


In [8]:
df

,ZIP Filename,Image Filename,Metadata,positive,negative,steps,POSITIVE,NEGATIVE,SETTING
95,./95641157_토군.zip,hanimix_real_v16B 서술형 와카로 찍먹 해봅니다._80.png,"parameters: (1female, solo:1.6, (Korean female...","parameters: (1female, solo:1.6, (Korean female...","Negative prompt:\n(easyNegative, ng_deepnegati...","Steps: 45, Sampler: DPM++\n2M SDE Karras, CFG ...","(1female, solo:1.6, (Korean female, korean mix...","(easyNegative, ng_deepnegative_v1_75t:1.2), (b...","Steps: 45, Sampler: DPM++ 2M SDE Karras, CFG s..."
422,./95684180_바코드.zip,잡샷 와이드_91.jpg,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,"Negative prompt: EasyNegative, (pubic\nhair:1....","Steps: 40, Sampler:\nDPM++ 2M Karras, CFG scal...","full_body, quiff,((little happy face)),light b...","EasyNegative, (pubic hair:1.5), puffy eyes, (f...","Steps: 40, Sampler: DPM++ 2M Karras, CFG scale..."
423,./95684180_바코드.zip,잡샷 와이드_85.jpg,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,"Negative prompt: EasyNegative, (pubic hair:1.5...","Steps:\n40, Sampler: DPM++ 2M Karras, CFG scal...","full_body, cornrows,((normal face)),dirty blon...","EasyNegative, (pubic hair:1.5), puffy eyes, (f...","Steps: 40, Sampler: DPM++ 2M Karras, CFG scale..."
424,./95684180_바코드.zip,잡샷 와이드_84.jpg,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,"Negative prompt: EasyNegative,\n(pubic hair:1....","Steps: 40, Sampler:\nDPM++ 2M Karras, CFG scal...","full_body, twin drills,((wondering face)),burg...","EasyNegative, (pubic hair:1.5), puffy eyes, (f...","Steps: 40, Sampler: DPM++ 2M Karras, CFG scale..."
425,./95684180_바코드.zip,잡샷 와이드_90.jpg,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,exif: ExifMM*_x0008__x0001_‡i_x0004__x0001__x0...,"Negative prompt: EasyNegative, (pubic\nhair:1....","Steps: 40, Sampler:\nDPM++ 2M Karras, CFG scal...","full_body, messy hair,((beaming_smile face)),m...","EasyNegative, (pubic hair:1.5), puffy eyes, (f...","Steps: 40, Sampler: DPM++ 2M Karras, CFG scale..."
...,...,...,...,...,...,...,...,...,...
8484,./95666114_비스비스.zip,평범한 인물샷_81.png,"parameters: instagram photo,goddess-like body,...","parameters: instagram photo,goddess-like body,...",Negative prompt: (CyberRealistic_Negative-neg:...,"Steps: 50, Sampler: DPM++ 2M Karras,\nCFG scal...","instagram photo,goddess-like body,real photos,...","(CyberRealistic_Negative-neg:0.8),mature,curvy...","Steps: 50, Sampler: DPM++ 2M Karras, CFG scale..."
8486,./95666114_비스비스.zip,평범한 인물샷_94.png,"parameters: instagram photo,goddess-like body,...","parameters: instagram photo,goddess-like body,...",Negative prompt: (CyberRealistic_Negative-neg:...,"Steps: 50, Sampler:\nDPM++ SDE Karras, CFG sca...","instagram photo,goddess-like body,real photos,...","(CyberRealistic_Negative-neg:0.8),mature,curvy...","Steps: 50, Sampler: DPM++ SDE Karras, CFG scal..."
8491,./95666114_비스비스.zip,평범한 인물샷_87.png,"parameters: instagram photo,goddess-like body,...","parameters: instagram photo,goddess-like body,...",Negative prompt: (CyberRealistic_Negative-neg:...,"Steps: 50, Sampler:\nDPM++ 2M Karras, CFG scal...","instagram photo,goddess-like body,real photos,...","(CyberRealistic_Negative-neg:0.8),mature,curvy...","Steps: 50, Sampler: DPM++ 2M Karras, CFG scale..."
8494,./95666114_비스비스.zip,평범한 인물샷_86.png,"parameters: instagram photo,goddess-like body,...","parameters: instagram photo,goddess-like body,...",Negative prompt: (CyberRealistic_Negative-neg:...,"Steps: 50, Sampler:\nDPM++ 2M Karras, CFG scal...","instagram photo,goddess-like body,real photos,...","(CyberRealistic_Negative-neg:0.8),mature,curvy...","Steps: 50, Sampler: DPM++ 2M Karras, CFG scale..."


In [9]:
sample_excel = os.path.join(download_path, "sample_0107.xlsx")
df.to_excel(sample_excel, index=False)